# Nuz Klassifir

### Download NLTK packages

In [ ]:
import nltk
from sklearn import metrics
#nltk.download()

### Training Data Collection and Preprocessing

### Test Data Collection and Preprocessing

In [2]:
import json
json_file = open('test\posts.json', encoding='utf8')
json_reader = json.load(json_file)

articles = []
for user in json_reader.values():
    for article in user:
        #use the title and description as input
        articles.append(article[0]+' '+ article[1])
        
print(articles[0])

الداخلية المصرية تُعلن تفاصيل حادث النزهة القاهرة - مباشر: أعلنت وزارة الداخلية المصرية، تفاصيل استشهاد النقيب ماجد أحمد عبد الرازق من قوة وحدة مباحث قسم شرطة النزهة وإصابة القوة المرافقة له، اليوم الأحد. وقالت الوزارة في بيان اليوم، أنه أثناء مرور النقيب ماجد أحمد عبد الرازق محمد، من قوة …
